<a href="https://colab.research.google.com/github/geoskimoto/SNOTEL_ErrorAndOutlier_Tools/blob/main/TemperatureOutliers_04_06_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library and Data Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import zscore 
from functools import reduce

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/My Drive/30yrNorm_Review/Data

/content/gdrive/My Drive/30yrNorm_Review/Data


In [ ]:
ls

OR_1991-2021_temps.xlsx  WA_1991-2021_temps.xlsx
OR_WY1990-2020.xlsx      WA_WY1990-2020.xlsx


In [ ]:
WA_Sites = pd.read_excel('WA_1991-2021_temps.xlsx')
# pd.to_datetime(WA_Sites['Date'])
WA_Sites.set_index('Date',inplace=True)
# WA_Sites.sample(5)

OR_Sites = pd.read_excel('OR_1991-2021_temps.xlsx')
# pd.to_datetime(OR_Sites['Date'])
OR_Sites.set_index('Date',inplace=True)
# OR_Sites.sample(5)

ORWA_Sites = pd.merge(OR_Sites,WA_Sites,left_index=True,right_index=True)
ORWA_Sites.head(1)

,Aneroid Lake #2 (302) Air Temperature Average (degF),Aneroid Lake #2 (302) Air Temperature Maximum (degF),Aneroid Lake #2 (302) Air Temperature Minimum (degF),Annie Springs (1000) Air Temperature Average (degF),Annie Springs (1000) Air Temperature Maximum (degF),Annie Springs (1000) Air Temperature Minimum (degF),Anthony Lake (1245) Air Temperature Average (degF),Anthony Lake (1245) Air Temperature Maximum (degF),Anthony Lake (1245) Air Temperature Minimum (degF),Arbuckle Mtn (304) Air Temperature Average (degF),Arbuckle Mtn (304) Air Temperature Maximum (degF),Arbuckle Mtn (304) Air Temperature Minimum (degF),Bald Mtn AM OR (447) Air Temperature Average (degF),Bald Mtn AM OR (447) Air Temperature Maximum (degF),Bald Mtn AM OR (447) Air Temperature Minimum (degF),Barney Creek (1246) Air Temperature Average (degF),Barney Creek (1246) Air Temperature Maximum (degF),Barney Creek (1246) Air Temperature Minimum (degF),Bear Flat Meadow AM (1291) Air Temperature Average (degF),Bear Flat Meadow AM (1291) Air Temperature Maximum (degF),Bear Flat Meadow AM (1291) Air Temperature Minimum (degF),Bear Grass (1166) Air Temperature Average (degF),Bear Grass (1166) Air Temperature Maximum (degF),Bear Grass (1166) Air Temperature Minimum (degF),Beaver Reservoir (331) Air Temperature Average (degF),Beaver Reservoir (331) Air Temperature Maximum (degF),Beaver Reservoir (331) Air Temperature Minimum (degF),Big Red Mountain (341) Air Temperature Average (degF),Big Red Mountain (341) Air Temperature Maximum (degF),Big Red Mountain (341) Air Temperature Minimum (degF),Big Sheep AM (1229) Air Temperature Average (degF),Big Sheep AM (1229) Air Temperature Maximum (degF),Big Sheep AM (1229) Air Temperature Minimum (degF),Bigelow Camp (343) Air Temperature Average (degF),Bigelow Camp (343) Air Temperature Maximum (degF),Bigelow Camp (343) Air Temperature Minimum (degF),Billie Creek Divide (344) Air Temperature Average (degF),Billie Creek Divide (344) Air Temperature Maximum (degF),Billie Creek Divide (344) Air Temperature Minimum (degF),Blazed Alder (351) Air Temperature Average (degF),...,Stampede Pass (788) Air Temperature Minimum (degF),Stevens Pass (791) Air Temperature Average (degF),Stevens Pass (791) Air Temperature Maximum (degF),Stevens Pass (791) Air Temperature Minimum (degF),Surprise Lakes (804) Air Temperature Average (degF),Surprise Lakes (804) Air Temperature Maximum (degF),Surprise Lakes (804) Air Temperature Minimum (degF),Swamp Creek (975) Air Temperature Average (degF),Swamp Creek (975) Air Temperature Maximum (degF),Swamp Creek (975) Air Temperature Minimum (degF),Swift Creek (1012) Air Temperature Average (degF),Swift Creek (1012) Air Temperature Maximum (degF),Swift Creek (1012) Air Temperature Minimum (degF),Thunder Basin (817) Air Temperature Average (degF),Thunder Basin (817) Air Temperature Maximum (degF),Thunder Basin (817) Air Temperature Minimum (degF),Tinkham Creek (899) Air Temperature Average (degF),Tinkham Creek (899) Air Temperature Maximum (degF),Tinkham Creek (899) Air Temperature Minimum (degF),Touchet (824) Air Temperature Average (degF),Touchet (824) Air Temperature Maximum (degF),Touchet (824) Air Temperature Minimum (degF),Trinity (1171) Air Temperature Average (degF),Trinity (1171) Air Temperature Maximum (degF),Trinity (1171) Air Temperature Minimum (degF),Trough (832) Air Temperature Average (degF),Trough (832) Air Temperature Maximum (degF),Trough (832) Air Temperature Minimum (degF),Upper Wheeler (841) Air Temperature Average (degF),Upper Wheeler (841) Air Temperature Maximum (degF),Upper Wheeler (841) Air Temperature Minimum (degF),Waterhole (974) Air Temperature Average (degF),Waterhole (974) Air Temperature Maximum (degF),Waterhole (974) Air Temperature Minimum (degF),Wells Creek (909) Air Temperature Average (degF),Wells Creek (909) Air Temperature Maximum (degF),Wells Creek (909) Air Temperature Minimum (degF),White Pass E.S. (863) Air Temperature Average (degF),White Pass E.S. (863) Air Temperature Ma

In [ ]:
ORWA_Tavg = ORWA_Sites[ORWA_Sites.columns[::3]]
stations = [ORWA_Tavg.columns[i].rsplit(' ', 4)[0] for i in range(0, len(ORWA_Tavg.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Air Temperature Average (degF)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Tavg.columns = header

ORWA_Tmax = ORWA_Sites[ORWA_Sites.columns[1::3]]
stations = [ORWA_Tmax.columns[i].rsplit(' ', 4)[0] for i in range(0, len(ORWA_Tmax.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Air Temperature Maximum (degF)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Tmax.columns = header
# ORWA_Precip_Incr.columns = stations

ORWA_Tmin = ORWA_Sites[ORWA_Sites.columns[2::3]]
stations = [ORWA_Tmin.columns[i].rsplit(' ', 4)[0] for i in range(0, len(ORWA_Tmin.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Air Temperature Minimum (degF)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Tmin.columns = header

ORWA_Sites2 = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [ORWA_Tavg,ORWA_Tmax,ORWA_Tmin])
ORWA_Sites2.head(1)

station,Aneroid Lake #2 (302),Annie Springs (1000),Anthony Lake (1245),Arbuckle Mtn (304),Bald Mtn AM OR (447),Barney Creek (1246),Bear Flat Meadow AM (1291),Bear Grass (1166),Beaver Reservoir (331),Big Red Mountain (341),Big Sheep AM (1229),Bigelow Camp (343),Billie Creek Divide (344),Blazed Alder (351),Blue Mountain Spring (357),Bourne (361),Bowman Springs (362),Buckskin Lake AM (1292),Call Meadows AM (1289),Cascade Summit (388),Chemult Alternate (395),Clackamas Lake (398),Clear Lake (401),Cold Springs Camp (406),County Line (422),Cox Flat AM (1273),Crazyman Flat (1010),Daly Lake (434),Derr. (440),Diamond Lake (442),Eilertson Meadows (464),Emigrant Springs (470),Finley Corrals AM (1274),Fish Creek (477),Fish Lk. (479),Fourmile Lake (483),Gerber Reservoir (945),Gold Center (494),Government Corrals (1250),Greenpoint (504),...,Mowich (941),Mt. Tebo (1126),Muckamuck (1259),Olallie Meadows (672),Paradise (679),Park Creek Ridge (681),Pepper Creek (1104),Pigtail Peak (692),Pinto Rock (1263),Pope Ridge (699),Potato Hill (702),Quartz Peak (707),Rainy Pass (711),Rex River (911),Salmon Meadows (728),Sasse Ridge (734),Satus Pass (1231),Sawmill Ridge (1068),Sentinel Butte (1043),Sheep Canyon (748),Skate Creek (1257),Skookum Creek (912),Sourdough Gulch (985),Spencer Meadow (776),Spirit Lake (777),Spruce Springs (984),Stampede Pass (788),Stevens Pass (791),Surprise Lakes (804),Swamp Creek (975),Swift Creek (1012),Thunder Basin (817),Tinkham Creek (899),Touchet (824),Trinity (1171),Trough (832),Upper Wheeler (841),Waterhole (974),Wells Creek (909),White Pass E.S. (863)
parameter,Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),Air Temperature Average (degF),...,Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature Minimum (degF),Air Temperature M

## Helper Functions: Z Score, Boxplots, Plotting Timeseries with Z Score Outliers

In [ ]:
#Make sure to choose/change correct file path for intended parameter before mass running with for loop.

def zScore(station_of_interest, parameter_of_interest, degrees_of_freedom):
  # global df, df_outliers
  df = pd.DataFrame(ORWA_Sites2.loc[:, (station_of_interest, parameter_of_interest)])
  zs = zscore(df, ddof=degrees_of_freedom, nan_policy='omit')
  df.insert(loc=0, column='zscore', value=zs)
  # return df.sort_values('zscore', ascending=True)[:top_scores]

  df_outliers = df[(df['zscore'] < -3) | (df['zscore'] > 3)].head(45)
  return df_outliers
  # df_outliers.to_excel('./TempOutliers/AvgTempOutliers/' + f'{parameter_of_interest}' + ' ZScoreOutliers ' + f'{station_of_interest}' + '.xlsx')

In [ ]:
def boxplots(station, data_pt_type):

  params = ['Air Temperature Average (degF)','Air Temperature Maximum (degF)','Air Temperature Minimum (degF)']

  df = [ORWA_Sites2[station][params[i]] for i in range(len(params))]

  fig = go.Figure()
  for i in range(len(params)):
  #   WA = WA_Sites[WA_Sites[i] != 0].dropna()

    fig.add_trace(go.Box(y = df[i], 
            name = params[i],
            boxpoints=data_pt_type, #options include: 'all', 'outliers','suspectedoutliers'
            hovertext = df[i].index,
            jitter=0.2,
            whiskerwidth=0.5                
    ))

  fig.update_layout(
          autosize=False,
          width=1200,
          height=500,
          margin=dict(l=20, r=20, t=2, b=8),
          paper_bgcolor="LightSteelBlue",
          title={
          'text': f'{station}',
          'y':0.98,
          'x':0.45,
          'xanchor': 'center',
          'yanchor': 'top'
          }
  )

  fig.show()

In [ ]:
def plotData(stations, parameter_of_interest):

  fig = go.Figure()
 
  for k in stations:

    fig.add_trace(go.Scatter(
      x=ORWA_Sites2.index,
      y=ORWA_Sites2[k][parameter_of_interest],
      mode='lines',
      name=k,
      hovertext = ORWA_Sites2[k][parameter_of_interest],   
  ))

  #   fig.add_trace(go.Scatter(
  #     x=df_outliers.index,
  #     y=df_outliers[k][parameter_of_interest],
  #     mode='markers',
  #     name=k,
  #     hovertext = df_outliers.index,   
  # ))

  fig.update_xaxes(title_text= 'Date')
  fig.update_yaxes(title_text= parameter_of_interest)

  fig.update_layout(
   height=800,
   width=1100,
  )

  fig.show()

In [ ]:
def zScore_plot(station_of_interest, parameter_of_interest, degrees_of_freedom):
  # global df, df_outliers
  df = pd.DataFrame(ORWA_Sites2.loc[:, (station_of_interest, parameter_of_interest)])
  zs = zscore(df, ddof=degrees_of_freedom, nan_policy='omit')
  df.insert(loc=0, column='zscore', value=zs)
  # return df.sort_values('zscore', ascending=True)[:top_scores]

  df_outliers = df[(df['zscore'] < -3) | (df['zscore'] > 3)].head(45)
  # return df_outliers
  # df_outliers.to_excel('./TempOutliers/AvgTempOutliers/' + f'{parameter_of_interest}' + ' ZScoreOutliers ' + f'{station_of_interest}' + '.xlsx')

  fig = go.Figure()
 
  fig.add_trace(go.Scatter(
    x=ORWA_Sites2.index,
    y=ORWA_Sites2[station_of_interest][parameter_of_interest],
    mode='lines',
    name=station_of_interest,
    hovertext = ORWA_Sites2[station_of_interest][parameter_of_interest],   
  ))

  fig.add_trace(go.Scatter(
    x=df_outliers.index,
    y=df_outliers[station_of_interest][parameter_of_interest],
    mode='markers',
    name=station_of_interest,
    hovertext = df_outliers.index,   
  ))

  fig.update_xaxes(title_text= 'Date')
  fig.update_yaxes(title_text= parameter_of_interest)

  fig.update_layout(
    height=800,
    width=1100,
  )

  fig.show()

## Analysis of Sites

In [ ]:
scores  = zScore ('Spirit Lake (777)', 'Air Temperature Average (degF)',2)
scores.head(5)

,zscore,Spirit Lake (777)
,,Air Temperature Average (degF)
Date,,
1990-07-11,3.097455,80.0
1990-09-11,5.246987,105.0
1990-09-12,3.871286,89.0
1990-09-13,4.473155,96.0
1990-09-14,4.043249,91.0


In [ ]:
boxplots('White Pass E.S. (863)', 'outliers')

In [ ]:
# station_of_interest = 'Aneroid Lake #2 (302)'
params = ['Air Temperature Average (degF)', 'Air Temperature Maximum (degF)', 'Air Temperature Minimum (degF)']

for i in params:
  zScore_plot(station_of_interest='Spirit Lake (777)',parameter_of_interest = i, degrees_of_freedom = 2)